# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [9]:
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
MODEL = "llama3.2"

def get_links(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/Qwen/Qwen2.5-Omni-7B',
 '/manycore-research/SpatialLM-Llama-1B',
 '/ds4sd/SmolDocling-256M-preview',
 '/ByteDance/InfiniteYou',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/ByteDance/InfiniteYou-FLUX',
 '/spaces/Stable-X/Hi3DGen',
 '/spaces/Qwen/Qwen2.5-Omni-7B-Demo',
 '/spaces/VAST-AI/TripoSG',
 '/spaces',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/a-m-team/AM-DeepSeek-R1-Distilled-1.4M',
 '/datasets/glaiveai/reasoning-v1-20m',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/PixelAI-Team/TalkBody4D',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'About page', 'url': 'https://huggingface.co/'},
  {'type': 'Company page', 'url': 'https://discuss.huggingface.co/brand'},
  {'type': 'Careers/Jobs page',
   'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'Products Models page', 'url': 'https://huggingface.co/models'},
  {'type': 'Datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'Spaces Research Lab page',
   'url': 'https://ui.endpoints.huggingface.co'},
  {'type': 'Blog page', 'url': 'https://blog.huggingface.co/'},
  {'type': 'GitHub Repository page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter Page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn Company Page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [39]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        try:
            result += f"\n\n{link['type']}\n"
            result += Website(link["url"]).get_contents()
        except Exception as e:
            print('*** ', 'this page(', link, ')is error: ', e, '***')
            pass
    return result

In [37]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'company page', 'url': 'https://brand.huggingface.co/'}, {'type': ' careers/jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': ' blog', 'url': 'https://blog.huggingface.co/'}, {'type': 'research and publications', 'url': 'https://discuss.huggingface.co/'}, {'type': 'github repository', 'url': 'https://github.com/huggingface'}]}
***  this page( {'type': 'company page', 'url': 'https://brand.huggingface.co/'} )is error:  HTTPSConnectionPool(host='brand.huggingface.co', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x11b2555d0>: Failed to resolve 'brand.huggingface.co' ([Errno 8] nodename nor servname provided, or not known)"))
***  this page( {'type': ' blog', 'url': 'https://blog.huggingface.co/'} )is error:  HTTPSConnectionPool(host='blog.huggingface.co', port=443): Max retries exceeded with url: / (C

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [40]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co/'}, {'type': 'Company/Blog', 'url': 'https://blog.huggingface.co/'}, {'type': 'Join/Discord', 'url': 'https://join.discord.com/huggingface'}, {'type': 'Learn', 'url': 'https://huggingface.co/learn'}]}
***  this page( {'type': 'Company/Blog', 'url': 'https://blog.huggingface.co/'} )is error:  HTTPSConnectionPool(host='blog.huggingface.co', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x11c29fad0>: Failed to resolve 'blog.huggingface.co' ([Errno 8] nodename nor servname provided, or not known)")) ***
***  this page( {'type': 'Join/Discord', 'url': 'https://join.discord.com/huggingface'} )is error:  HTTPSConnectionPool(host='join.discord.com', port=443): Max retries exceeded with url: /huggingface (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x11b1436d0>: Failed to resolve 'join.discord.com' ([Errno 8] n

'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-V3-0324\nUpdated\n5 days ago\n•\n86.6k\n•\n2.17k\nQwen/Qwen2.5-Omni-7B\nUpdated\n1 day ago\n•\n53k\n•\n981\nmanycore-research/SpatialLM-Llama-1B\nUpdated\n11 days ago\n•\n12.6k\n•\n838\nds4sd/SmolDocling-256M-preview\nUpdated\n9 days ago\n•\n57.8k\n•\n1.09k\nByteDance/InfiniteYou\nUpdated\n7 days ago\n•\n511\nBrowse 1M+ models\nSpaces\nRunning\n1.11k\n1.11k\nDeepSite\n🐳\nG

In [43]:
def create_brochure(company_name, url):
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [44]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co/'}, {'type': 'Home/Brand page', 'url': 'https://huggingface.co'}, {'type': 'Careers/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Blog', 'url': 'https://discuss.huggingface.co'}]}


# Hugging Face Brochure

[Cover Image: A group of people from diverse backgrounds working together on a computer]

**Welcome to Hugging Face**

Hugging Face is the AI community building the future. We empower machine learning professionals, researchers, and businesses to collaborate on projects that transform industries and society. Our platform provides a suite of tools for text, image, video, audio, and 3D models, ensuring a seamless experience across all modalities.

**Building a Community**

We believe in collaboration and community-driven innovation. We host unlimited public models, datasets, and applications, allowing users to share their work, build portfolios, and connect with like-minded professionals. Our platform is not just about tools – it's about building a global network of ML enthusiasts who come together to advance the field.

**What Sets Us Apart**

* **Unparalleled Model Size**: Access over 1 million models, updated daily, covering a wide range of applications, from natural language processing to computer vision.
* **Enterprise-Grade Security**: Our platform offers enterprise-grade security features, including access controls, dedicated support, and priority logins for large organizations.
* **Multi-Modal Experience**: Explore and utilize text, image, video, audio, and 3D models in a single interface.
* **Continuous Innovation**: Stay up-to-date with the latest research advancements and developments in our open-source stack.

**Join the Movement**

Whether you're an AI expert, researcher, or simply a curious individual looking to explore the world of machine learning, Hugging Face is here to inspire and empower. Join us today, and become part of a community that's shaping the future.

* **Sign up now** and experience the full potential of our platform
* **Explore our resources**, including tutorials, guides, and documentation
* **Connect with our community**, share your work, and learn from others

[Call-to-Action Button: Join Now]

**Our Values**

At Hugging Face, we're driven by a few core values:

* **Community**: We believe in collaboration and community-driven innovation.
* **Innovation**: We're committed to developing cutting-edge tools that advance the field of machine learning.
* **Accessibility**: Our platform is designed for everyone, regardless of background or experience.

**Our Team**

Meet our team of AI enthusiasts, researchers, and developers who are passionate about empowering the next generation of machine learning professionals.

[Team Members' Photos and Bios]

#HuggingFace #AICommunity #MachineLearning

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [48]:
def stream_brochure(company_name, url):
    stream = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [49]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co'}, {'type': 'Company/Enterprise', 'url': '/enterprise'}, {'type': 'Careers/Jobs', 'url': 'https://apply.workable.com/huggingface/'}, {'type': '', 'url': 'https://huggingface.co/docs/'}, {'type': 'Blog', 'url': 'https://blog.huggingface.co'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
***  this page( {'type': 'Company/Enterprise', 'url': '/enterprise'} )is error:  Invalid URL '/enterprise': No scheme supplied. Perhaps you meant https:///enterprise? ***
***  this page( {'type': 'Blog', 'url': 'https://blog.huggingface.co'} )is error:  HTTPSConnectionPool(host='blog.huggingface.co', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x11cbc94d0>: Failed to resolve 'blog.huggingface.co'

Hugging Face Brochure
=====================

**About Us**
----------------

Hugging Face is the leading platform for the machine learning community, building the foundation of AI tooling with open-source technology. Our mission is to make it easy for anyone to build and train AI models.

**Our Story**
-----------------

We began our journey in 2016, founded by Francis Reinhart and Evan Jhurkeya, focusing on making transformer-based machine learning models widely available. Today, we have grown into a global community of over 50,000 organizations, providing a platform for collaboration, model sharing, and innovation.

**Our Culture**
-----------------

At Hugging Face, we value:

*   **Community**: We believe in the power of collective collaboration to drive AI progress.
*   **Innovation**: We're committed to developing cutting-edge AI technologies that make a real impact.
*   **Transparency**: Our open-source approach ensures that everyone can access and contribute to our developments.
*   **Humility**: We recognize that AI is a rapidly evolving field, and we're always eager to learn from others.

**Using AI in Real-World Applications**
------------------------------------------

Our platform provides an extensive range of applications for:

*   Text generation
*   Image recognition
*   Audio processing
*   3D geometry generation

These tools can be used to create innovative solutions across industries, such as:

*   Healthcare
*   Finance
*   Education
*   Entertainment

**What Our Partners Say**
-----------------------------

We're proud to have partnered with prominent companies like Meta, Amazon, Google, Intel, and Microsoft.

**Join Us**
--------------

If you're passionate about creating cutting-edge AI solutions, we invite you to:

*   Explore our platform and see how AI can be used in your field.
*   Join the community and contribute to open-source developments.
*   Get career opportunities with our team of expert engineers and researchers.

### [Get Started](https://huggingface.co/login)

Join us on this exciting journey and shape the future of AI together!

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>